Import video

In [2]:
from imutils import contours 
from skimage import measure  
import numpy as np          
import argparse
import imutils
import matplotlib.pyplot as plt
import cv2

vidcap = cv2.VideoCapture('VID_20220518_205445.mp4')

success, image = vidcap.read()
count = 1
while success:
  cv2.imwrite("resources/image_%d.jpg" % count, image)    
  success, image = vidcap.read()
  count += 1

print("Saved", count, "images")

Saved 1265 images


Process video

In [3]:
circles = []

for i in range(1, count):
    image_address = "./resources/image_" + str(i) + ".jpg"

    ## read the image
    image = cv2.imread(image_address)

    ## convert BGR to RGB, we won't use this in any of the processes, just to test image loading
    rgb_image = cv2.cvtColor(image , cv2.COLOR_BGR2RGB) 

    ## convert BGR to greyscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    ## blur the image with a gussian filter to remove any noise, and also to soften the image
    blurred = cv2.GaussianBlur(gray, (11, 11), 0)

    # ## plot images in a nice array. We have to define the colormap for imshow to get a black and white image, as the default 
    # ## color map shows colors between yellow (pixel = 255) and black (pixel = 0)
    # fig , ax = plot_imshow_in_array([rgb_image , gray , blurred] , \
    #                                 max_image_in_row = 4 , \
    #                                 plots_kwargs_list = [{} , {'cmap':'gray', 'vmin':0, 'vmax':255} , {'cmap':'gray', 'vmin':0, 'vmax':255}] )

    # ## use this command to place enough spacing between the borders of images and the numbers
    # fig.tight_layout()

    thresh = cv2.threshold(blurred, 50, 255 , cv2.THRESH_BINARY)[1]

    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=4)

    MIN_ALLOWED_PIXEL_IN_REGION = 5000
    MAX_GREY_VALUE = 255

    ## find out regions and label them
    labels = measure.label(thresh, connectivity=2, background=0)


    ## The list holding all contiguous masks that will be found
    individual_masks = []

    ## loop over the unique components
    for label in np.unique(labels):
        # if this is the background (dark) label, ignore it
        if label == 0:
            continue
            
        # otherwise, construct the label mask and count the number of pixels 
        labelMask = np.zeros(thresh.shape, dtype="uint8")
        labelMask[labels == label] = MAX_GREY_VALUE
        numPixels = cv2.countNonZero(labelMask)

        
        # this condition filters "very" small regions
        if numPixels > MIN_ALLOWED_PIXEL_IN_REGION:
            #plt.figure()        
            individual_masks.append(labelMask)
    
    ## make a copy of the individual_mask to avoid them from being overwritten
    individual_masks_cp = [mask.copy() for mask in individual_masks]

    value = 0
    max_circle = 0

    for idx , current_mask in enumerate(individual_masks_cp):
        cnts = cv2.findContours(current_mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
    #     cnts = contours.sort_contours(cnts)[0]

        ## create a mask to hold the values of the enclosing circle, we also fill inside the cirlce. 
        filled_circle = np.zeros(current_mask.shape)

        ## find the coordinates of the center and the radius of the circle
        ((cX, cY), radius) = cv2.minEnclosingCircle(cnts[0])
        
        ## fill the mask values surrounded by the circle
        cv2.circle(filled_circle, (int(cX), int(cY)), int(radius),(255, 255, 255), -1)
        
        ## count the number of pixels in the circle
        circle_area = cv2.countNonZero(filled_circle)
        
        ## count the number of non-zero pixels in the original region
        mask_non_zero_area = cv2.countNonZero(individual_masks[idx])
        
        ## plot the circle and the region, and show the filled ratio in each circle
        # plot_imshow_in_array([filled_circle , individual_masks[idx]] , max_image_in_row = 4)

        if ((mask_non_zero_area / circle_area) > max_circle):
            max_circle = mask_non_zero_area / circle_area

        #plt.title("circles' filled ratio = {:.2}".format(mask_non_zero_area / circle_area))

    circles.append(max_circle)

Find bit width

In [4]:
import copy

bit_list = []
for circle in circles:
    if circle > 0.1 and circle <= 0.9:
        bit_list.append(2)
    elif circle > 0.9:
        bit_list.append(1)
    else:
        bit_list.append(0)

cnt_dict = {}

last = -1
cnt = 0
count = 0
for b in bit_list:
    count += 1
    if last != -1 and b != last:
        if cnt not in cnt_dict:
            cnt_dict[cnt] = 1
        else:
            cnt_dict[cnt] += 1
        cnt = 0
    last = b
    cnt += 1

filter = []

for k in cnt_dict.keys():
    if cnt_dict[k] == 1:
        filter.append(k)

for k in filter:
    del cnt_dict[k]

sum = 0
cnt = 0
for k, v in cnt_dict.items():
    sum += k * v
    cnt += v

bit_width = sum//cnt

print("V2:", bit_width)

bit_width = 0
cnt = 0
for k1, v1 in cnt_dict.items():
    if v1 > cnt:
        cnt = v1
        bit_width = k1

print(cnt_dict.items())

print("V1:", bit_width)


V2: 3
dict_items([(3, 201), (6, 93), (1, 21), (2, 16), (5, 4)])
V1: 3


Decode message

In [26]:
import crcmod

def match(seq, pattern):
    for i in range(0, len(seq)):
        if seq[i] != pattern[i] and seq != 2:
            return False
    return True

def compress(seq):
    count0 = 0
    count1 = 0
    for val in seq:
        if val == 0:
            count0 += 1
        else:
            count1 += 1
    if count0 > count1:
        return 0
    else:
        return 1

def demod(seq):
    result = 0
    for i in range(0, 16):
        if i % 2 == 0:
            if seq[i] == 0 and seq[i+1] == 1:
                result = result | (1<<(7- i//2))
    return result

def crc16(data : bytearray, offset , length):
    if data is None or offset < 0 or offset > len(data)- 1 and offset+length > len(data):
        return 0
    crc = 0x0
    for i in range(0, length):
        crc ^= data[offset + i] << 8
        for j in range(0,8):
            if (crc & 0x8000) > 0:
                crc =(crc << 1) ^ 0x1021
            else:
                crc = crc << 1
    return crc & 0xFFFF

# Create preamble pattern
pattern = []

val = 1
for i in range(0, 16):
    for i in range(0, bit_width):
        pattern.append(val)
    if val == 1:
        val = 0
    else:
        val = 1

print(bit_list[450:500])

# Search for preamble
start = 0
for i in range(0, len(bit_list)-len(pattern)):
    if match(bit_list[i:i+len(pattern)], pattern):
        print("Found preamble starting at", i)
        start = i+len(pattern)
        break
        

# Compress list into bits
mod_message = []
for i in range(start, len(bit_list)):
    if (i - start) % bit_width == 0:
        mod_message.append(compress(bit_list[i:i+bit_width]))

# Read length
mess_length = demod(mod_message[:16])

if mess_length < 20 or mess_length > 40:
    print("Invalid message size")
else:
    payload = []
    for i in range(0, mess_length):
        payload.append(chr(demod(mod_message[(i+1)*16:(i+2)*16])))

    crc_data = [int("".join(map(str, mod_message[i:i+8])), 2) for i in range(16, (mess_length * 2 + 2) * 8, 8)]
    
    calc_crc = crc16(crc_data, 0, len(crc_data))
    rx_crc = demod(mod_message[(mess_length * 2 + 2) * 8:(mess_length * 2 + 4) * 8 ]) << 8
    rx_crc = rx_crc | demod(mod_message[(mess_length * 2 + 4) * 8:(mess_length * 2 + 6) * 8 ])

    if calc_crc == rx_crc:
        print(payload)
    else:
        print("Incorrect crc")


[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0]
Found preamble starting at 14
['H', 'e', 'l', 'l', 'o', ' ', 'V', 'L', 'C', '&', 'S', ' ', '2', '0', '2', '1', '-', '2', '0', '2', '2']
